### Import Necessary Libraries

In [1]:
## For Web Scraping
from selenium import webdriver              #### webdriver links up with the browserand performs the necessary actions.
from selenium.webdriver.common.keys import Keys         #### Keys is a class that contains keys that we can use to simulate keystrokes.
from selenium.webdriver.chrome.options import Options
import time

## For Data Analysis
import pandas as pd

## For String Matching
from fuzzywuzzy import fuzz, process

pd. set_option("display.max_rows", None)        ## Display all the rows in the DataFrame

/Users/eben.emmanuel/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Loading the Required Data

In [2]:
## Obtain the Company Names
Companyname = pd.read_csv('company_data.csv')
Companyname.dropna(inplace=True)
Companyname.head()

,CompanyName
0,mercusys
1,tp-link
2,futura
3,vestige
4,msi


### Web Scraping Using Selenium

In [3]:
## Open the website
def start():
    global company_name, driver
    PATH = '/Users/eben.emmanuel/Downloads/chromedriver'

    chrome_options = Options()
    chrome_options.add_argument("--headless")       ## To run the browser in headless mode. To avoid pop-up window.
    
    #### Webbrowser is Chrome and the webdriver is in the Path specified.
    driver = webdriver.Chrome(executable_path= PATH, chrome_options= chrome_options)
    
    ## Input Company Name from DataFrame
    for name in Companyname['CompanyName']:
        #### Webbrowser is Chrome and the webdriver is in the Path specified.
        driver = webdriver.Chrome(executable_path=PATH, chrome_options=chrome_options)
        
        company_name = name
        company_name = company_name.replace(' ','+')

        driver.get(f'https://www.google.com/search?q={company_name.lower()}')
        main()


def main():
    try:
        for article in driver.find_elements_by_xpath('//div[@ class= "jtfYYd"]'):
            try:
                link = article.find_element_by_xpath('.//div[@ class= "TbwUpd NJjxre"]/cite').text
                name = article.find_element_by_xpath('.//h3[@ class= "LC20lb MBeuO DKV0Md"]').text
            except:
                pass

            variables = ["VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf", "VwiC3b MUxGbd yDYNvb lyLwlc", "VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc"]
            for i in variables:
                try:
                    content = article.find_element_by_xpath(f'.//div[@ class= "{i}"]').text
                except:
                    pass
            
            f.write('\n' + company_name.replace('+',' ') + ',' + link.replace(',','|') + ',' + name.replace(',','|') + ',' + content.replace(',','|'))

        for article in driver.find_elements_by_xpath('//div[@ class= "tF2Cxc"]'):
            link = article.find_element_by_xpath('.//div[@ class= "TbwUpd NJjxre"]/cite').text
            name = article.find_element_by_xpath('.//h3[@ class= "LC20lb MBeuO DKV0Md"]').text
            content = article.find_element_by_xpath('.//div[@ class= "IsZvec"]').text
            
            f.write('\n' + company_name.replace('+',' ') + ',' + link.replace(',','|') + ',' + name.replace(',','|') + ',' + content.replace(',','|'))

    except Exception as e:
        print(e)

    else:
        ## Close the browser
        driver.close()

if __name__ == '__main__':
    f = open('scraped_data.csv', 'w')
    f.write('CompanyName, Link, PageName, Summary')
    start()

/var/folders/k3/yy8qff1s4sj0k71g9rcc339w8gvs2x/T/ipykernel_2099/598182960.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path= PATH, chrome_options= chrome_options)
/var/folders/k3/yy8qff1s4sj0k71g9rcc339w8gvs2x/T/ipykernel_2099/598182960.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path= PATH, chrome_options= chrome_options)
/var/folders/k3/yy8qff1s4sj0k71g9rcc339w8gvs2x/T/ipykernel_2099/598182960.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=PATH, chrome_options=chrome_options)
/var/folders/k3/yy8qff1s4sj0k71g9rcc339w8gvs2x/T/ipykernel_2099/598182960.py:15: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=PATH, chrome_options=chrome_options)
/var/folders/k3/yy8qff1s4sj0k71g9rcc339w8gvs2x/T/ipyke

In [4]:
f.close()

#### View the Scraped Data

In [44]:
import csv
df = pd.read_csv('scraped_data.csv',  quoting=csv.QUOTE_NONE, encoding='utf-8')

### Remove the Duplicates
df.drop_duplicates(subset= [' Summary'], inplace= True)
df.dropna(inplace= True)

df.reset_index(inplace= True)   ### Reset the Index

df.drop(columns= 'index', inplace= True)
df.head()

,CompanyName,Link,PageName,Summary
0,mercusys,https://www.mercusys.com › support,Support-Welcome to MERCUSYS,MERCUSYS Simulator. View the web management in...
1,mercusys,https://www.flipkart.com › routers › mercusys~...,Buy Mercusys Routers Online at Best Prices in ...,Buy Mercusys Routers Online| check Mercusys Ro...
2,mercusys,https://www.tatacliq.com › mercusys › page-1,Mercusys - Tata CLiQ,Mercusys · MERCUSYS MW325R 300Mbps Wireless N ...
3,mercusys,https://www.mercusys.co.in,Welcome to MERCUSYS,MERCUSYS is a networking brand which dedicates...
4,mercusys,https://www.amazon.in › Mercusys-MW305R-Wirele...,Buy Mercusys MW305R 300Mbps Wireless Wi-Fi WiF...,MERCUSYS is a sub brand of TP-LINK and is rela...


### Obtaining the Score

<h5> METHOD 1: Using fuzz.extract() </h5>

In [45]:
### Converting to List
Company = df['CompanyName'].unique().tolist()

### Making a copy of the DataFrame
df1 = df.copy()

In [46]:
## This is a faster method
df1['Scores'] = 0

for i in range(len(Company)):       ##### Iterate through the whole DataFrame
    try:
        scores = process.extract(Company[i], df1[' Summary'])    #### Obtain the scores for each Company with Summary.
        for x in range(len(df1[' Summary'])):
            df1['Scores'][scores[x][2]] = scores[x][1]       #### Since we obtain the index as output
        else:
            df1['Scores'][i] = 0       #### Convert all other indexes to 0
    except:
        pass

/var/folders/k3/yy8qff1s4sj0k71g9rcc339w8gvs2x/T/ipykernel_2099/1655463531.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Scores'][scores[x][2]] = scores[x][1]       #### Since we obtain the index as output


In [47]:
### Updated DataFrame
df1

,CompanyName,Link,PageName,Summary,Scores
0,mercusys,https://www.mercusys.com › support,Support-Welcome to MERCUSYS,MERCUSYS Simulator. View the web management in...,60
1,mercusys,https://www.flipkart.com › routers › mercusys~...,Buy Mercusys Routers Online at Best Prices in ...,Buy Mercusys Routers Online| check Mercusys Ro...,57
2,mercusys,https://www.tatacliq.com › mercusys › page-1,Mercusys - Tata CLiQ,Mercusys · MERCUSYS MW325R 300Mbps Wireless N ...,60
3,mercusys,https://www.mercusys.co.in,Welcome to MERCUSYS,MERCUSYS is a networking brand which dedicates...,60
4,mercusys,https://www.amazon.in › Mercusys-MW305R-Wirele...,Buy Mercusys MW305R 300Mbps Wireless Wi-Fi WiF...,MERCUSYS is a sub brand of TP-LINK and is rela...,60
5,mercusys,https://play.google.com › store › apps › detai...,MERCUSYS – Apps on Google Play,The MERCUSYS app allows you to quickly and eas...,0
6,tp-link,http://tplinkwifi.net,Trying to configure the Router? | TP-Link,Trying to configure the Router? It looks like ...,60
7,tp-link,https://en.wikipedia.org › wiki › TP-Link,TP-Link - Wikipedia,(Chinese: 普联技术; Pinyin: pǔ lián jìshù)| is a g...,57
8,tp-link,https://play.google.com › store › apps › detai...,TP-Link Tether – Apps on Google Play,TP-Link Tether provides the easiest way to acc...,60
9,tp-link,https://www.flipkart.com › routers › tp-link~b...,Upto 70% off on Tp-link WiFi Routers | Flipkar...,TP-Link is a Chinese brand that was started in...,60


In [48]:
## Data with Scores above 80
above_80 = df1[df1['Scores'] > 80]
above_80

,CompanyName,Link,PageName,Summary,Scores
42,procdna,https://in.linkedin.com › ...,Harpreet Singh - Partner - ProcDNA | LinkedIn,Harpreet Singh. Partner at ProcDNA | We are hi...,86
49,srinidhi electricals,https://www.justdial.com › Bangalore › Srinidh...,Srinidhi Electricals and repairs in Bangalore ...,Srinidhi Electricals and repairs in Bangalore....,90
50,srinidhi electricals,https://in.linkedin.com › srinidhielectricals-...,srinidhielectricals Melavanki - Class-1|electr...,srinidhielectricals Melavanki. Class-1|electri...,90
51,srinidhi electricals,https://www.indiamart.com › srinidhi-electricals,Srinidhi Electricals - IndiaMART,Srinidhi Electricals - Service Provider of Fan...,90
52,srinidhi electricals,https://www.indiamart.com › srinidhi-electrica...,Srinidhi Electricals Hard Ware - Wholesaler of...,Srinidhi Electricals Hard Ware38 1st Main Road...,90
65,daiji world,https://www.instagram.com › daijiworldnews,Daijiworld.com (@daijiworldnews) • Instagram p...,48.6k Followers| 1452 Following| 8017 Posts - ...,86
77,sri nandi publications,https://www.justdial.com › Bangalore › Sri-Nan...,Sri Nandi Publications in Padmanabhanagar| Ban...,Sri Nandi Publications in Padmanabhanagar| Ban...,90
78,sri nandi publications,https://www.mypustak.com › publication › books...,BUY Sri Nandi Publications-AT INDIA'S SOCIALLY...,Shop the quality assured free used books of Sr...,90
79,sri nandi publications,https://nicelocal.in › 🚩 Bangalore › 🛒 Shops i...,Sri Nandi Publications - Shops in Bangalore - ...,Sri Nandi Publications can be found by the fol...,90
80,sri nandi publications,http://lrc.acharyainstitutes.in › koha › opac-...,Results of search for 'Provider:Sri Nandi Publ...,Material type: Text; Format: print ; Literary ...,90


<h5> METHOD 2: Using fuzz.partial_ratio() </h5>

The partial ratio() function allows us to perform substring matching. This works by taking the shortest string and matching it with all substrings that are of the same length. This function is also useful when matching names.

In [32]:
### Making a copy of the DataFrame
df2 = df.copy()

In [33]:
## This is a Faster and more Efficient Method
df2['Scores'] = 0

for i in range(len(df2['CompanyName'])):
    try:
        scores = fuzz.partial_ratio(df2['CompanyName'][i].lower(), df2[' Summary'][i].lower())
        df2['Scores'][i] = scores
    except:
        pass

/var/folders/k3/yy8qff1s4sj0k71g9rcc339w8gvs2x/T/ipykernel_2099/4163344959.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Scores'][i] = scores


In [34]:
### Updated DataFrame
df2

,CompanyName,Link,PageName,Summary,Scores
0,mercusys,https://www.mercusys.com › support,Support-Welcome to MERCUSYS,MERCUSYS Simulator. View the web management in...,100
1,mercusys,https://www.flipkart.com › routers › mercusys~...,Buy Mercusys Routers Online at Best Prices in ...,Buy Mercusys Routers Online| check Mercusys Ro...,100
2,mercusys,https://www.tatacliq.com › mercusys › page-1,Mercusys - Tata CLiQ,Mercusys · MERCUSYS MW325R 300Mbps Wireless N ...,100
3,mercusys,https://www.mercusys.co.in,Welcome to MERCUSYS,MERCUSYS is a networking brand which dedicates...,100
4,mercusys,https://www.amazon.in › Mercusys-MW305R-Wirele...,Buy Mercusys MW305R 300Mbps Wireless Wi-Fi WiF...,MERCUSYS is a sub brand of TP-LINK and is rela...,100
5,mercusys,https://play.google.com › store › apps › detai...,MERCUSYS – Apps on Google Play,The MERCUSYS app allows you to quickly and eas...,100
6,tp-link,http://tplinkwifi.net,Trying to configure the Router? | TP-Link,Trying to configure the Router? It looks like ...,100
7,tp-link,https://en.wikipedia.org › wiki › TP-Link,TP-Link - Wikipedia,(Chinese: 普联技术; Pinyin: pǔ lián jìshù)| is a g...,100
8,tp-link,https://play.google.com › store › apps › detai...,TP-Link Tether – Apps on Google Play,TP-Link Tether provides the easiest way to acc...,100
9,tp-link,https://www.flipkart.com › routers › tp-link~b...,Upto 70% off on Tp-link WiFi Routers | Flipkar...,TP-Link is a Chinese brand that was started in...,100


In [35]:
## Data with Scores above 80
above_80 = df2[df2['Scores'] > 80]
above_80

,CompanyName,Link,PageName,Summary,Scores
0,mercusys,https://www.mercusys.com › support,Support-Welcome to MERCUSYS,MERCUSYS Simulator. View the web management in...,100
1,mercusys,https://www.flipkart.com › routers › mercusys~...,Buy Mercusys Routers Online at Best Prices in ...,Buy Mercusys Routers Online| check Mercusys Ro...,100
2,mercusys,https://www.tatacliq.com › mercusys › page-1,Mercusys - Tata CLiQ,Mercusys · MERCUSYS MW325R 300Mbps Wireless N ...,100
3,mercusys,https://www.mercusys.co.in,Welcome to MERCUSYS,MERCUSYS is a networking brand which dedicates...,100
4,mercusys,https://www.amazon.in › Mercusys-MW305R-Wirele...,Buy Mercusys MW305R 300Mbps Wireless Wi-Fi WiF...,MERCUSYS is a sub brand of TP-LINK and is rela...,100
5,mercusys,https://play.google.com › store › apps › detai...,MERCUSYS – Apps on Google Play,The MERCUSYS app allows you to quickly and eas...,100
6,tp-link,http://tplinkwifi.net,Trying to configure the Router? | TP-Link,Trying to configure the Router? It looks like ...,100
7,tp-link,https://en.wikipedia.org › wiki › TP-Link,TP-Link - Wikipedia,(Chinese: 普联技术; Pinyin: pǔ lián jìshù)| is a g...,100
8,tp-link,https://play.google.com › store › apps › detai...,TP-Link Tether – Apps on Google Play,TP-Link Tether provides the easiest way to acc...,100
9,tp-link,https://www.flipkart.com › routers › tp-link~b...,Upto 70% off on Tp-link WiFi Routers | Flipkar...,TP-Link is a Chinese brand that was started in...,100


<h5> METHOD 3: Using fuzz.extractOne() </h5>

In [36]:
df.head()

,CompanyName,Link,PageName,Summary
0,mercusys,https://www.mercusys.com › support,Support-Welcome to MERCUSYS,MERCUSYS Simulator. View the web management in...
1,mercusys,https://www.flipkart.com › routers › mercusys~...,Buy Mercusys Routers Online at Best Prices in ...,Buy Mercusys Routers Online| check Mercusys Ro...
2,mercusys,https://www.tatacliq.com › mercusys › page-1,Mercusys - Tata CLiQ,Mercusys · MERCUSYS MW325R 300Mbps Wireless N ...
3,mercusys,https://www.mercusys.co.in,Welcome to MERCUSYS,MERCUSYS is a networking brand which dedicates...
4,mercusys,https://www.amazon.in › Mercusys-MW305R-Wirele...,Buy Mercusys MW305R 300Mbps Wireless Wi-Fi WiF...,MERCUSYS is a sub brand of TP-LINK and is rela...


In [37]:
names = []  
names = pd.DataFrame(names) 
for i in df:
    if process.extractOne(i,df) != None: 
        string = process.extractOne(i,df)[0]

        for index, val in enumerate(df['CompanyName']):
            if val.lower() in str(df[' Summary'][index]).lower():    ## In case of some nan values. Also names need to be in lower to match the summary.
                string = df[' Summary'][index]
                head = df[' PageName'][index]
                link = df[' Link'][index]
                score = process.extractOne(i,df)[1]
                    
                names = names.append({'score':score,"companyname":val,"string":string,"head":head,"link":link}, ignore_index=True) 
            else:
                names = names.append({'score':0,"companyname":val,"string":None,"head":None,"link":None}, ignore_index=True) 

In [38]:
names

,score,companyname,string,head,link
0,60.0,mercusys,MERCUSYS Simulator. View the web management in...,Support-Welcome to MERCUSYS,https://www.mercusys.com › support
1,60.0,mercusys,Buy Mercusys Routers Online| check Mercusys Ro...,Buy Mercusys Routers Online at Best Prices in ...,https://www.flipkart.com › routers › mercusys~...
2,60.0,mercusys,Mercusys · MERCUSYS MW325R 300Mbps Wireless N ...,Mercusys - Tata CLiQ,https://www.tatacliq.com › mercusys › page-1
3,60.0,mercusys,MERCUSYS is a networking brand which dedicates...,Welcome to MERCUSYS,https://www.mercusys.co.in
4,60.0,mercusys,MERCUSYS is a sub brand of TP-LINK and is rela...,Buy Mercusys MW305R 300Mbps Wireless Wi-Fi WiF...,https://www.amazon.in › Mercusys-MW305R-Wirele...
5,60.0,mercusys,The MERCUSYS app allows you to quickly and eas...,MERCUSYS – Apps on Google Play,https://play.google.com › store › apps › detai...
6,60.0,tp-link,Trying to configure the Router? It looks like ...,Trying to configure the Router? | TP-Link,http://tplinkwifi.net
7,60.0,tp-link,(Chinese: 普联技术; Pinyin: pǔ lián jìshù)| is a g...,TP-Link - Wikipedia,https://en.wikipedia.org › wiki › TP-Link
8,60.0,tp-link,TP-Link Tether provides the easiest way to acc...,TP-Link Tether – Apps on Google Play,https://play.google.com › store › apps › detai...
9,60.0,tp-link,TP-Link is a Chinese brand that was started in...,Upto 70% off on Tp-link WiFi Routers | Flipkar...,https://www.flipkart.com › routers › tp-link~b...


- From all the above methods, we can see that the accurate results were from METHOD 2 and METHOD 1.